### Library

In [3]:
from datetime import datetime, timedelta
import oandapy
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import pickle
import sys
import os
import talib
sys.path.append('/Users/toshio/MyProject/fx')
from config import token

### Oandaのインスタンス化

In [4]:
access_token = token
oanda = oandapy.API(environment="practice", access_token=access_token)

### 過去データの取得

In [52]:
i = 0
end = datetime(2018, 8, 1, 0, 0)
res = []
while i < 2:
    start = end - timedelta(days = 4999)
    print(i, start, end)

    end_day =  '{0}-{1:02d}-{2:02d}-{3:02d}:{4:02d}:{5:02d}'.format(end.year, end.month, end.day, end.hour, end.minute, end.second)
    start_day =  '{0}-{1:02d}-{2:02d}-{3:02d}:{4:02d}:{5:02d}'.format(start.year, start.month, start.day, start.hour, start.minute, start.second)

    res.append(oanda.get_history(instrument="USD_JPY",granularity="D",start = start_day, end = end_day))
    i += 1
    end = start - timedelta(days = 1)

0 2004-11-23 00:00:00 2018-08-01 00:00:00
1 1991-03-17 00:00:00 2004-11-22 00:00:00


In [53]:
dfs = []
dfs.append(pd.DataFrame(res[0]['candles']))
dfs.append(pd.DataFrame(res[1]['candles']))
df = pd.concat([dfs[1], dfs[0]], axis = 0)

In [54]:
df = df.drop(['complete'], axis = 1)

df['time'] = df['time'].str[:-8]
df['time'] = df['time'].str.replace('T',' ')
times = [datetime.strptime(v, '%Y-%m-%d %H:%M:%S') for v in df['time']]
df['time'] = times
df = df.set_index('time',drop = True)

### 出力

In [60]:
with open('../intermediate_data/data.pickle', mode='wb') as f:
    pickle.dump(df, f)